In [ ]:
pip install "pycozo[embedded,pandas]"

In [ ]:
from pycozo.client import Client
import pandas as pd
from pycozo.client import Client
client = Client('rocksdb', 'datafile.db')

In [ ]:
# create reified edges "hasEmployee" between  organisation and person
import pandas as pd

# 1) Charger les relations existantes
edge_label_df = pd.DataFrame(client.run('?[id_e, ln] := *edge_label[id_e, ln]'))
edge_df       = pd.DataFrame(client.run('?[id_e, ns, nd] := *edge[id_e, ns, nd]'))

# 2) Ne garder que les arêtes workAt
df = edge_label_df[edge_label_df['ln'] == 'workAt'].merge(edge_df, on='id_e')

# 3) Générer IDs pour hasEmployee à partir de 16631283
next_re = 16631283
e_comp_edge = []   # [id_re, id_e]
e_comp_node = []   # [id_re, id_n]
new_edges   = []   # [id_e_re, ns', nd']
new_labels  = []   # [id_e_re, "hasEmployee"]

for _, row in df.iterrows():
    rid = next_re
    next_re += 1

    # a) rattacher l'arête workAt
    e_comp_edge.append([rid, row['id_e']])
    # b) rattacher les deux nœuds : person (ns) et organisation (nd)
    e_comp_node.append([rid, row['ns']])
    e_comp_node.append([rid, row['nd']])
    # c) dans edge[] la nouvelle arête réifiée : inverser ns/nd
    new_edges.append([rid, row['nd'], row['ns']])
    # d) associer le label hasEmployee
    new_labels.append([rid, "hasEmployee"])

# 4) Fonction utilitaire pour formater une commande Cozo
def format_cozo(rel, rows, cols):
    entries = ",".join(
        "[" + ",".join(f'"{x}"' if isinstance(x, str) else str(x) for x in row) + "]"
        for row in rows
    )
    return f"?[{','.join(cols)}] <- [{entries}] :put {rel} {{{','.join(cols)}}}"

# 5) Afficher les commandes Cozo à exécuter
print(format_cozo("e_composed_of_edge", e_comp_edge, ["id_re","id_e"]))
print(format_cozo("e_composed_of_node", e_comp_node, ["id_re","id_n"]))
print(format_cozo("edge",               new_edges,   ["id_e","ns","nd"]))
print(format_cozo("edge_label",         new_labels,  ["id_e","ln"]))


In [ ]:
res = client.run(format_cozo("e_composed_of_edge", e_comp_edge, ["id_re","id_e"]))
res

In [ ]:
res = client.run(format_cozo("e_composed_of_node", e_comp_node, ["id_re","id_n"]))
res

In [ ]:
res = client.run(format_cozo("edge",               new_edges,   ["id_e","ns","nd"]))
res

In [ ]:
res = client.run(format_cozo("edge_label",         new_labels,  ["id_e","ln"]))
res